In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [89.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hi

In [2]:
# Install great expectations
!pip install great_expectations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 30.6 MB/s 
     |████████████████████████████████| 4.1 MB 53.6 MB/s 
     |████████████████████████████████| 9.9 MB 31.1 MB/s 
     |████████████████████████████████| 793 kB 65.2 MB/s 
     |████████████████████████████████| 139 kB 63.5 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 109 kB 67.8 MB/s 
     |████████████████████████████████| 381 kB 62.4 MB/s 
     |████████████████████████████████| 423 kB 49.2 MB/s 
     |████████████████████████████████| 127 kB 40.6 MB/s 
     |████████████████████████████████| 546 kB 78.1 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstallin

In [3]:
# Import SparkSession and great expectations 
from pyspark.sql import SparkSession
import great_expectations as ge

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# Start Spark session
from pyspark import SparkFiles

In [4]:
# Read the data into a Spark DataFrame.
employees_df = spark.read.csv('/content/attrition.csv', sep=',', header=True)
employees_df.show()

+-----+---+---------+-----------------+---------+----------+---------+-------+----------------+-----------------------+------+----------+--------------+--------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+-------------+---------------+
|   id|Age|Attrition|   BusinessTravel|DailyRate|Department|Education|JobRole|DistanceFromHome|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|StandardHours|WorkLifeBalance|
+-----+---+---------+-----------------+---------+----------+---------+-------+----------------+-----------------------+------+----------+--------------+--------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+-------------+---------------+
|12378| 35|      Yes|Travel_Frequently|    

In [5]:
# Save the column names to a list. 
columns = employees_df.schema.names
print(columns)

['id', 'Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department', 'Education', 'JobRole', 'DistanceFromHome', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'StandardHours', 'WorkLifeBalance']


In [6]:
# Create the Great Expectations DataFrame
employees_df_ge = ge.dataset.SparkDFDataset(employees_df)
employees_df_ge.head()

,id,Age,Attrition,BusinessTravel,DailyRate,Department,Education,JobRole,DistanceFromHome,EnvironmentSatisfaction,...,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,StandardHours,WorkLifeBalance
0,12378,35,Yes,Travel_Frequently,502,S,a,l,5,4,...,Divorced,9200,10448,2,Y,Yes,5,1,80,1
1,12379,42,No,Non-Travel,497,S,a,l,3,3,...,Single,10719,9049,3,Y,No,7,2,80,4
2,12380,34,No,Non-Travel,505,T,e,c,4,3,...,Single,9843,9941,3,Y,No,10,3,80,1
3,12381,43,No,Non-Travel,493,T,e,c,5,2,...,Divorced,12077,10150,5,Y,Yes,1,3,80,4
4,12382,43,Yes,Travel_Rarely,536,R,e,s,4,4,...,Single,10486,10008,2,Y,No,9,1,80,3


In [7]:
# Test that each column doesn't contain null values. If a column does contain null values print out the column name.
results = []
for column in columns:
  results.append(employees_df_ge.expect_column_values_to_not_be_null(column=column)['success'])
false_index = results.index(False)
print(columns[false_index])

Over18


In [8]:
# Use a function to remove the null values and bad data from a temporary view and save the data with null values to a parquet file. 
def clean_data():
  employees_df.createOrReplaceTempView('employees')
  employee_null_df = spark.sql(
      """
      SELECT
        *
      FROM employees
      WHERE Over18 IS NULL 
            OR Attrition == "Yes" 
            OR StandardHours < 80
            OR Age < 18
      """
  )
  employee_null_df.createOrReplaceTempView('removed_employees')
  employee_null_df.write.mode("overwrite").parquet("removed_employees_parquet")
  print("Bad records found and written to a temporary view, 'removed_employees', and parquet , 'removed_employees_parquet'.")

In [9]:
# Retrieve the "success" fields for the columns that meet the criteria in the clean_data() function.
result = []
result.append(employees_df_ge.expect_column_values_to_not_be_null('Over18')['success'])
result.append(employees_df_ge.expect_column_values_to_be_in_set('Attrition', ['Yes'])['success'])
result.append(employees_df_ge.expect_column_values_to_be_between('StandardHours', 0, 80)['success'])
result.append(employees_df_ge.expect_column_values_to_be_between('Age', 0, 18)['success'])
# If the columns meet the criteria call the clean_data() function.
if False in result:
  clean_data()
else:
  print("success")

Bad records found and written to a temporary view, 'removed_employees', and parquet , 'removed_employees_parquet'.


In [10]:
# Read in our customers parquet data
removed_employees_df=spark.read.parquet('removed_employees_parquet')
removed_employees_df.show(truncate=False)
# Get the number of rows. 
print(removed_employees_df.count())

+-----+---+---------+-----------------+---------+----------+---------+-------+----------------+-----------------------+------+----------+--------------+--------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+-------------+---------------+
|id   |Age|Attrition|BusinessTravel   |DailyRate|Department|Education|JobRole|DistanceFromHome|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|StandardHours|WorkLifeBalance|
+-----+---+---------+-----------------+---------+----------+---------+-------+----------------+-----------------------+------+----------+--------------+--------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+-------------+---------------+
|12378|35 |Yes      |Travel_Frequently|502 

In [13]:
# Use a function to transform the employees temporary view if the id is not in the removed_employees.
def transform_data():
  transformed_df = spark.sql(
      """
      SELECT
        id,
        Attrition,
        BusinessTravel,
        DailyRate,
        Department,
        JobRole,
        HourlyRate,
        MonthlyIncome,
        MonthlyRate,
        OverTime,
        PercentSalaryHike,
        PerformanceRating,
        StandardHours,
        UNIX_TIMESTAMP() AS Created_At,
        UNIX_TIMESTAMP() AS Updated_at,
        'From Client' AS Source
      FROM employees
      WHERE id not in (select id from removed_employees)
      """
  )
  return transformed_df

In [14]:
# Call the transformation function and convert the transformed DataFrame into a Great Expectations DataFrame
transformed_df = transform_data()
transformed_df_ge = ge.dataset.SparkDFDataset(transformed_df)
# Retrieve the "success" fields for the transformed DataFrame.
transformed_result = []
transformed_result.append(transformed_df_ge.expect_table_column_count_to_equal(16)['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('Source')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('Created_At')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_not_be_null('Updated_At')['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_be_in_set('Attrition', ['No'])['success'])
transformed_result.append(transformed_df_ge.expect_column_values_to_be_between('StandardHours', 80, 150)['success'])
# If the "success" field is 'False' print "Failed", if not then write the transformed DataFrame to a parquet file.
if False in transformed_result:
  print('Failed')
else:
  transformed_df.write.mode("overwrite").parquet("employee_parquet")
  print("Success: Wrote the transformed data to a parquet file, 'employee_parquet'")


Success: Wrote the transformed data to a parquet file, 'employee_parquet'


In [15]:
# Read in our customers parquet data
employees_df=spark.read.parquet('employee_parquet')
employees_df.show(truncate=False)
# Get the number of rows. 
print(employees_df.count())

+-----+---------+-----------------+---------+----------+-------+----------+-------------+-----------+--------+-----------------+-----------------+-------------+----------+----------+-----------+
|id   |Attrition|BusinessTravel   |DailyRate|Department|JobRole|HourlyRate|MonthlyIncome|MonthlyRate|OverTime|PercentSalaryHike|PerformanceRating|StandardHours|Created_At|Updated_at|Source     |
+-----+---------+-----------------+---------+----------+-------+----------+-------------+-----------+--------+-----------------+-----------------+-------------+----------+----------+-----------+
|12379|No       |Non-Travel       |497      |S         |l      |50        |10719        |9049       |No      |7                |2                |80           |1660330941|1660330941|From Client|
|12380|No       |Non-Travel       |505      |T         |c      |46        |9843         |9941       |No      |10               |3                |80           |1660330941|1660330941|From Client|
|12381|No       |Non-Trav